In [3]:
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.
/home/praneet/venv/no_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/praneet/venv/no_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/praneet/venv/no_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/praneet/venv/no_gpu/lib/python3.6/site-pa

In [4]:
def smoothL1(y_true, y_pred):
    x = K.abs(y_true - y_pred)
    if K._BACKEND == 'tensorflow':
        x = tf.where(x < HUBER_DELTA, 0.5 * x ** 2, HUBER_DELTA * (x - 0.5 * HUBER_DELTA))
        return  K.sum(x)

In [5]:
train_csv = pd.read_csv('dataset/training_frames_keypoints.csv').T
test_csv = pd.read_csv('dataset/test_frames_keypoints.csv').T

In [6]:
train_csv.head()

,0,1,2,3,4,5,6,7,8,9,...,3452,3453,3454,3455,3456,3457,3458,3459,3460,3461
Unnamed: 0,Luis_Fonsi_21.jpg,Lincoln_Chafee_52.jpg,Valerie_Harper_30.jpg,Angelo_Reyes_22.jpg,Kristen_Breitweiser_11.jpg,Albert_Brooks_12.jpg,Daniel_Radcliffe_02.jpg,Laura_Bozzo_41.jpg,James_Hoffa_51.jpg,Christopher_Walken_20.jpg,...,Alma_Powell_42.jpg,Kweisi_Mfume_51.jpg,Kurt_Warner_10.jpg,Priyanka_Chopra_51.jpg,Charlotte_Casiraghi_11.jpg,Thomas_Ferguson_02.jpg,Valerie_Harper_00.jpg,Maggie_Smith_00.jpg,Erin_Hershey_Presley_41.jpg,Rocco_Buttiglione_42.jpg
0,45,41,56,61,58,67,35,90,47,126,...,68,34,59,122,59,35,72,52,82,93
1,98,83,69,80,94,134,57,68,101,107,...,78,73,152,93,71,94,133,149,91,134
2,47,43,56,58,58,69,35,88,47,126,...,65,34,66,119,59,35,76,56,82,93
3,106,91,77,95,104,144,62,74,117,122,...,87,82,178,105,84,105,147,160,105,147


In [7]:
img_row = 224
img_col = 224

In [8]:
all_coords = []
images = []
for i in range(train_csv.shape[1]):
    
    image_name = 'dataset/training/' + train_csv[i][0]
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (224, 224))
    images.append(image)
    
    # should be int
    coordinates = list(train_csv[i])[1:]
    all_coords.append(np.array(coordinates))
x_train = np.array(images)
del images
x_train = x_train.astype('float32')
x_train /= 255
x_train = np.expand_dims(x_train, axis=4)
y_train = np.array(all_coords)
del all_coords

/home/praneet/venv/no_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [9]:
all_coords = []
images = []
for i in range(test_csv.shape[1]):
    
    image_name = 'dataset/test/' + test_csv[i][0]
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (224, 224))
    images.append(image)
    
    coordinates = list(test_csv[i])[1:]
    all_coords.append(np.array(coordinates))
x_test = np.array(images)
del images
x_test = x_test.astype('float32')
x_test /= 255
x_test = np.expand_dims(x_test, axis=4)
y_test = np.array(all_coords)
del all_coords

/home/praneet/venv/no_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [10]:
x_test.shape

(770, 224, 224, 1)

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), strides=(1, 1),
                 activation='relu', input_shape=(img_row, img_col, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(312, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(136))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 128)       73856     
__________

In [12]:
model.compile(optimizer = 'adam',loss = 'mean_squared_error', metrics = ['mae','acc'])
model.fit(x_train,y_train,batch_size=32, epochs=5,validation_data=(x_test,y_test))

Train on 3462 samples, validate on 770 samples
Epoch 1/5
3462/3462 [==============================] - 592s 171ms/step - loss: 2656.4367 - mean_absolute_error: 38.9427 - acc: 0.0852 - val_loss: 3774.1611 - val_mean_absolute_error: 49.6517 - val_acc: 0.0182
Epoch 2/5
3462/3462 [==============================] - 618s 179ms/step - loss: 1561.1406 - mean_absolute_error: 30.3465 - acc: 0.1542 - val_loss: 3669.3370 - val_mean_absolute_error: 49.0197 - val_acc: 0.0182
Epoch 3/5
3462/3462 [==============================] - 617s 178ms/step - loss: 1428.7660 - mean_absolute_error: 29.0006 - acc: 0.1849 - val_loss: 3990.1832 - val_mean_absolute_error: 52.1912 - val_acc: 0.0273
Epoch 4/5
3462/3462 [==============================] - 732s 211ms/step - loss: 1355.6873 - mean_absolute_error: 28.3085 - acc: 0.2088 - val_loss: 1605.8291 - val_mean_absolute_error: 29.0624 - val_acc: 0.6182
Epoch 5/5
3462/3462 [==============================] - 798s 231ms/step - loss: 1302.6240 - mean_absolute_error: 27.59

In [28]:
image = cv2.imread('dataset/test/Aleksander_Kwasniewski_21.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (224, 224))
image = image / 255
image = np.expand_dims(image, axis=4)
image = np.expand_dims(image, axis=0)

# image = image.reshape(-1, 1)

result = model.predict(image)
image.shape

/home/praneet/venv/no_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """


(1, 224, 224, 1)

In [31]:
print(result.shape)

(1, 136)
